Auto-install Missing Ones

In [7]:
# Install dependencies (only if not already installed)
!pip install torch torchvision torchaudio torchsummary matplotlib scikit-learn --quiet



Imports & Dataset Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
# Path to dataset
data_dir = "AxialDataset"  # adjust path if needed

# Image transforms (force grayscale → resize → normalize)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),   # ✅ grayscale
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load datasets
train_dataset = datasets.ImageFolder(f"{data_dir}/Training", transform=transform)
test_dataset  = datasets.ImageFolder(f"{data_dir}/Testing",  transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

classes = train_dataset.classes
print("Classes:", classes)
print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
Train samples: 3272
Test samples: 786


In [4]:
# Counts from your dataset
class_counts = [1893, 602, 562, 215]  # [notumor, glioma, meningioma, pituitary]

# Inverse frequency
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
class_weights = class_weights.to(device)
print("Class Weights:", class_weights)


Class Weights: tensor([0.0005, 0.0017, 0.0018, 0.0047], device='cuda:0')


Visualize a Few Samples

In [5]:
model = models.resnet18(pretrained=True)

# Modify input layer (grayscale → 1 channel)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify output layer (num_classes = 4)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(classes))

model = model.to(device)


c:\Users\adith\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\adith\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = models.resnet18(pretrained=True)

# Modify input layer (grayscale → 1 channel)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify output layer (num_classes = 4)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(classes))

model = model.to(device)


Define Model (ResNet18 → 4 classes)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    history = {"train_loss": [], "val_acc": []}

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        history["train_loss"].append(running_loss / len(train_loader))
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {running_loss/len(train_loader):.4f} | Val Acc: {val_acc:.4f}")

    return history


In [9]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

data_dir = "AxialDataset"  # <-- change if your dataset is elsewhere

# Train augmentation
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Validation/Test transforms (no augmentation)
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load datasets
train_dataset = datasets.ImageFolder(f"{data_dir}/Training", transform=train_transform)
test_dataset = datasets.ImageFolder(f"{data_dir}/Testing", transform=test_transform)

# Split train into train+val (80/20 split)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# ✅ DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names
classes = train_dataset.classes
print("Classes:", classes)


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [12]:
import torch.nn as nn
import torch.optim as optim

# Loss function (CrossEntropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam is good, you can try SGD too)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)


Epoch 1/10 | Train Loss: 0.4488 | Val Acc: 0.8504
Epoch 2/10 | Train Loss: 0.2400 | Val Acc: 0.7557
Epoch 3/10 | Train Loss: 0.1771 | Val Acc: 0.8656
Epoch 4/10 | Train Loss: 0.1462 | Val Acc: 0.8962
Epoch 5/10 | Train Loss: 0.1088 | Val Acc: 0.8702
Epoch 6/10 | Train Loss: 0.0846 | Val Acc: 0.9618
Epoch 7/10 | Train Loss: 0.0762 | Val Acc: 0.9466
Epoch 8/10 | Train Loss: 0.0957 | Val Acc: 0.9374
Epoch 9/10 | Train Loss: 0.0696 | Val Acc: 0.9725
Epoch 10/10 | Train Loss: 0.0452 | Val Acc: 0.9756


In [14]:
torch.save(model.state_dict(), "brain_tumor_model_v2.pth")
print("✅ Model saved as brain_tumor_model_v2.pth")


✅ Model saved as brain_tumor_model_v2.pth


In [1]:
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import cv2
import matplotlib.cm as cm

# ======================
# Load Trained Model
# ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model architecture (ResNet18 modified for grayscale)
model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 classes
model.load_state_dict(torch.load("brain_tumor_model_v2.pth", map_location=device))
model.to(device)
model.eval()

# Class labels
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# ======================
# Image Transform
# ======================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# ======================
# Grad-CAM Implementation
# ======================
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_layers()

    def hook_layers(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()

        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def generate_heatmap(self, input_tensor, class_idx=None):
        output = self.model(input_tensor)

        if class_idx is None:
            class_idx = torch.argmax(output)

        # Backpropagate for target class
        self.model.zero_grad()
        output[0, class_idx].backward()

        # Pool gradients
        pooled_gradients = torch.mean(self.gradients, dim=[0, 2, 3])
        activations = self.activations[0]

        for i in range(len(pooled_gradients)):
            activations[i, :, :] *= pooled_gradients[i]

        heatmap = torch.mean(activations, dim=0).cpu().numpy()
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1
        return heatmap

# Initialize Grad-CAM on the last conv layer of ResNet18
grad_cam = GradCAM(model, model.layer4[1].conv2)

# ======================
# Prediction + Heatmap Function
# ======================
def predict_and_heatmap(img_path):
    image = Image.open(img_path).convert("L")  # grayscale
    img_tensor = transform(image).unsqueeze(0).to(device)

    # Prediction
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
    pred_class = classes[predicted.item()]

    # Heatmap
    heatmap = grad_cam.generate_heatmap(img_tensor, predicted.item())
    heatmap = cv2.resize(heatmap, (image.size[0], image.size[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    img = np.array(image.convert("RGB"))
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    return pred_class, image, Image.fromarray(superimposed_img)

# ======================
# Tkinter UI
# ======================
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if not file_path:
        return

    # Prediction + Heatmap
    prediction, orig_img, heatmap_img = predict_and_heatmap(file_path)

    # Display original image
    img_resized = orig_img.resize((200, 200))
    img_tk = ImageTk.PhotoImage(img_resized)
    panel.config(image=img_tk)
    panel.image = img_tk

    # Display heatmap image
    heatmap_resized = heatmap_img.resize((200, 200))
    heatmap_tk = ImageTk.PhotoImage(heatmap_resized)
    panel_heatmap.config(image=heatmap_tk)
    panel_heatmap.image = heatmap_tk

    # Display prediction
    result_label.config(text=f"Prediction: {prediction}", font=("Arial", 14, "bold"))

# Create main window
root = tk.Tk()
root.title("Brain Tumor Detection with Heatmap")
root.geometry("500x500")

# UI Elements
btn = Button(root, text="Upload MRI Image", command=upload_image, font=("Arial", 12), bg="lightblue")
btn.pack(pady=10)

frame = tk.Frame(root)
frame.pack()

panel = Label(frame)  # Original image
panel.grid(row=0, column=0, padx=10)

panel_heatmap = Label(frame)  # Heatmap
panel_heatmap.grid(row=0, column=1, padx=10)

result_label = Label(root, text="", font=("Arial", 14))
result_label.pack(pady=10)

# Run Tkinter loop
root.mainloop()


: 